In [1]:
import cv2                 
import numpy as np         
import os                  
from random import shuffle
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
path = 'asl_alphabet_train'

IMG_SIZE = 96

training_data = []
label=0
for (dirpath,dirnames,filenames) in os.walk(path):
    for dirname in dirnames:
        for(direcpath,direcnames,files) in os.walk(path+"/"+dirname):
            for file in files:
                    actual_path=path+"/"+dirname+"/"+file
                    # label=label_img(dirname)
                    path1 =path+"/"+dirname+'/'+file
                    img=cv2.imread(path1,cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
                    training_data.append([np.array(img),label])
        label=label+1
shuffle(training_data)
np.save('train_data.npy', training_data)
print(training_data)

In [ ]:
IMG_SIZE = 96
LR = 1e-3
nb_classes=28
MODEL_NAME = 'handsign.model'

def one_hot_targets_(labels_dense,nb_classes):
    targets = np.array(Y).reshape(-1)
    print(targets)
    one_hot_targets = np.eye(nb_classes)[targets]
    return one_hot_targets


train_data = np.load('train_data.npy',allow_pickle=True,encoding="latin1")


train = train_data[:]
test = train_data[:100]

print('traindatlen:'+str(len(train)))
print('testdatalen:'+str(len(test)))


X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]
Y1=one_hot_targets_(Y,nb_classes)
# 
# print('max y'+str(max(Y)))
# print('min y'+str(min(Y)))
print('val y'+str(Y1))
print('len X:'+str(len(X)))
print('len Y:'+str(len(Y)))
test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]
test_y1=one_hot_targets_(test_y,nb_classes)
test_y=test_y1
Y=Y1
print('test_x:'+str(len(test_x)))
print('test_y:'+str(len(test_y)))
print('val y'+str(test_y1))

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 8, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 16, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, nb_classes, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=0.01, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

model.fit({'input': X}, {'targets': Y}, n_epoch=15, validation_set=({'input': test_x}, {'targets': test_y}), 
snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

model.save(MODEL_NAME)



score = model.evaluate(test_x, test_y)
print('Test accuarcy: %0.4f%%' % (score[0] * 100))